In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt #그래프
import seaborn as sns
import missingno as msno #결측치(missing data) 시각화

In [3]:
#pandas를 이용해 해당경로에서 데이터를 가져온다.
#시간데이터를 별도의옵션(timestamp)을 통해 설정한다.
df = pd.read_csv('/kaggle/input/london-bike-sharing-dataset/london_merged.csv', parse_dates = ['timestamp'])
df.head()  #df의 head 상위 5개를 불러와줘.

In [4]:
#데이터의 타입과 구조

print('데이터의 구조는:', df.shape)
print('데이터의 타입은:', df.dtypes)
print('데이터의 칼럼은:', df.columns)

In [5]:
df.isna().sum() #데이터들의 결측치(missing data)가 있는지 확인

In [6]:
msno.matrix(df)
plt.show()

In [7]:
#추가적인 변수(year, month, dayofweek, hour) 생성
df['year'] = df['timestamp'].dt.year #df의 timestamp열에서 년도를 추출해서 df year를 만들어넣겠다
df['month'] = df['timestamp'].dt.month
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['hour'] = df['timestamp'].dt.hour
df.head()

In [8]:
df['year'].value_counts()  #탐색적 분석_데이터 형성 개수

In [9]:
a, b = plt.subplots(1, 1, figsize=(10, 5))
sns.boxplot(df['year'], df['cnt'])

#년도별로 자전거 이용객들을 boxplot으로 나타냄
#색깔부분이 중위값, 맨아래가 min값, 위에 줄그어진부분이 max값
#max값을 벗어난 부분을 이상치라고 함
#17년에 들어서 이용객이 감소함

In [10]:
a, b = plt.subplots(1, 1, figsize=(10, 5))
sns.boxplot(df['month'], df['cnt'])

In [11]:
a, b = plt.subplots(1, 1, figsize=(10, 5))
sns.boxplot(df['dayofweek'], df['cnt'])

#토,일에 자전거 이용 감소

In [12]:
a, b = plt.subplots(1, 1, figsize=(10, 5))
sns.boxplot(df['hour'], df['cnt'])

In [13]:
# 그래프 함수 만들기

def plot_bar(data, feature):
    fig = plt.figure(figsize=(12, 3))
    sns.barplot(x=feature, y='cnt', data=data, palette='Set3', orient='v')

In [14]:
plot_bar(df, 'hour')

In [15]:
plot_bar(df, 'dayofweek')

In [16]:
#이상치 제거(아웃라이어 제거)=>정규분포에서 양끝값

def is_outliers(s):
    lower_limit = s.mean() - (s.std()*3) # s평균 - s표준편차*3
    upper_limit = s.mean() + (s.std()*3) 
    return ~s.between(lower_limit, upper_limit) #lower_limit와 upper_limit..상한값과 하안값 사이리턴

In [17]:
df_out = df[~df.groupby('hour')['cnt'].apply(is_outliers)] #시간대별로 이용객수를 outlier를 적용해서 이상치 제거된 값

print('이상치 제거전:', df.shape)
print('이상치 제거후:', df_out.shape)

In [18]:
df_out.dtypes

In [19]:
df_out['weather_code'] = df_out['weather_code'].astype('category')
df_out['season'] = df_out['season'].astype('category')
df_out['year'] = df_out['year'].astype('category')
df_out['month'] = df_out['month'].astype('category')
df_out['hour'] = df_out['hour'].astype('category')

In [20]:
df_out.dtypes

In [21]:
# dummy처리 => 0, 1과 같은 이진처리
df_out = pd.get_dummies(df_out, columns=['weather_code', 'season', 'year', 'month', 'hour'])
df_out.head()

In [22]:
df_out.shape #59개의 변수로 구성.. 종속변수(y) 그 이외는 독립변수(x)

In [23]:
#독립, 종속변수 분리
df_y = df_out['cnt'] #df_out의 cnt는 y로쓰고
df_x = df_out.drop(['timestamp', 'cnt'], axis=1) #timestamp와 cnt를 제외한것은 독립변수x로 쓰겠다.
#axis 1이면 열기준/ 0이면 가로축기준
df_x.head()

In [24]:
df_y.head()

In [25]:
#훈련용과 테스트용 데이터 구분하기 => 추후에 학습이 잘 되었는지 체크하기 위함
#독립변수중 훈련/테스트용  종속변수중 훈련/테스트용 분류
#tab키 누르면 자동완성됨

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, random_state=66, test_size=0.3, shuffle=False)

In [26]:
print('x_train의 구조는:', x_train.shape)
print('y_train의 구조는:', y_train.shape)

print('x_test의 구조는:', x_test.shape)
print('y_test의 구조는:', y_test.shape)

#17000개 중에서 7:3으로 해서 12000으로 학습하고 5000개로 테스트
#변수는 57개
#y_train은 cnt(이용객)값만 있으므로 뒤에 빈칸으로 나타난다.

In [27]:
#keras로 딥러닝 진행
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

In [28]:
model = Sequential()
model.add(Dense(units=160, activation='relu', input_dim=57))
model.add(Dense(units=60, activation='relu'))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=1, activation='linear'))

In [29]:
model.summary()

In [30]:
model.compile(loss='mae', optimizer='adam', metrics=['mae']) #adam:loss가 줄어드는 방향으로 최적
early_stopping = EarlyStopping(monitor='loss', patience=5, mode='min')
history = model.fit(x_train, y_train, epochs=50, batch_size=1, validation_split=0.1, callbacks=[early_stopping])
#훈련중에 overfitting 또는 가적합이 발생하지 않도록 10%를 검증용데이터로 분리
#훈련용 데이터만 학습한 나머지 test에서 좋지 않은 결과가 나오는 경우를 방지
#가적합이 이뤄지기 전에 EarlyStopping으로 멈춰준다.

#훈련용 데이터 12000개 중 검증용 1200개 뽑아서 50번 반복
#매번돌때마다 에러를 표시함.. loss(18000개에 대한 에러), val_loss(1200에 대해서 체크=>다시 올라가면 가적합. 5번 올라가면 멈춘다.)

In [31]:
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('loss 비교')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legent(['val_loss', 'loss'])
plt.show()

In [ ]:
y_predict = model.predict(x_test)